In [14]:
import json 
import requests
import time
import urllib.request
import math
import os
from collections import defaultdict


TOKEN = "743391112:AAF60UYlsEhkgb9qU-APK5nqxffhTb9LMbY"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)
URL_FILE = "https://api.telegram.org/file/bot{}/".format(TOKEN)


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    global tipekonten
    konten = [None,None]
    try:
        try:
            konten[0] = updates["result"][last_update]["message"]["photo"][2]["file_id"]
            konten[1] = updates["result"][last_update]["message"]["caption"]
        except IndexError:
            konten[0] = updates["result"][last_update]["message"]["photo"][0]["file_id"]
            konten[1] = updates["result"][last_update]["message"]["caption"]
        tipekonten="gambar"
    except KeyError:
        try:
            konten = updates["result"][last_update]["message"]["text"]
            tipekonten="text"
        except KeyError:
            try:
                konten[0] = updates["result"][last_update]["message"]["location"]["latitude"]
                konten[1] = updates["result"][last_update]["message"]["location"]["longitude"]
                tipekonten="location"
            except KeyError:
                True
    
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    message_id = updates["result"][last_update]["message"]["message_id"]
    return ( tipekonten,konten, chat_id, message_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
    
def send_file(file_id, chat_id):
    url = URL + "sendPhoto?photo={}&chat_id={}".format(file_id, chat_id)
    get_url(url)
    
def save_file(file_id, chat_id,path):
    url = URL + "getFile?file_id={}".format(file_id)
    js = get_json_from_url(url)
    file_path = js["result"]["file_path"]
    url = URL_FILE + file_path
    testfile = urllib.request.urlretrieve(url,str(path)+"/"+str(file_id)+".jpg")
    send_message("image downloaded",chat_id)
def save_text(text, chat_id,path, namafile):
    file = open(str(path)+"/"+namafile+".txt","w") 
    file.write(text)
    file.close()
    
def calc_distance(lat1, lon1):
    lat2=-6.229697
    lon2=106.816049
    R = 6371e3  #metres
    o1 = math.radians(lat1)
    o2 = math.radians(lat2)
    deltaφ = math.radians(lat2-lat1)
    deltaλ = math.radians(lon2-lon1)

    a = math.sin(deltaφ/2) * math.sin(deltaφ/2) + math.cos(o1) * math.cos(o2) * math.sin(deltaλ/2) * math.sin(deltaλ/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return(d)

def input_tiket(listData,konten,chat_id):
    notiket=str(konten[13:])
    chat_id=str(chat_id)
    listData[chat_id]['no_tiket']=notiket
    listData[chat_id]['gambar']=defaultdict(dict)
    listData[chat_id]['gambar']['sebelum']['status']=False
    listData[chat_id]['gambar']['sesudah']['status']=False
    listData[chat_id]['gambar']['sebelum']['file_id']=''
    listData[chat_id]['gambar']['sesudah']['file_id']=''
    listData[chat_id]['keterangan']=defaultdict(dict)
    listData[chat_id]['keterangan']['status']=False
    listData[chat_id]['keterangan']['isi']=''
    listData[chat_id]['location']=defaultdict(dict)
    listData[chat_id]['location']['status']=False
    listData[chat_id]['location']['latitude']=0
    listData[chat_id]['location']['longitude']=0
    listData[chat_id]['location']['distance']=0
    listData[chat_id]['statusTiket']=False
    send_message("Nomor Tiket Berhasil Terinput",chat_id)
    return(listData)

    
def input_gambar(listData, konten, chat_id):
    tipeGambar = str(konten[1][14:]).lower()
    chat_id=str(chat_id)
    try:
        if(tipeGambar=="sebelum"):
            listData[chat_id]['gambar']['sebelum']['status']=True
            listData[chat_id]['gambar']['sebelum']['file_id']=konten[0]
            send_message("Gambar tampak 'sebelum' berhasil terinput",chat_id)
        elif(tipeGambar=="sesudah"):
            listData[chat_id]['gambar']['sesudah']['status']=True
            listData[chat_id]['gambar']['sebelum']['file_id']=konten[0]
            send_message("Gambar tampak 'sesudah' berhasil terinput",chat_id)
    except KeyError:
        send_message("Anda belum menginputkan nomor tiket",chat_id)
    return(listData)    
def input_keterangan(listData, konten, chat_id):
    keterangan = str(konten[17:])
    chat_id=str(chat_id)
    try:
        listData[chat_id]['keterangan']['status']=True
        listData[chat_id]['keterangan']['isi']=keterangan 
        send_message("Keterangan berhasil terinput",chat_id)
    except KeyError:
        send_message("Anda belum menginputkan nomor tiket",chat_id)
    return(listData)
def input_location(listData, konten, chat_id):
    chat_id=str(chat_id)
    try:
        listData[chat_id]['location']['status']=True
        listData[chat_id]['location']['latitude']=konten[0]
        listData[chat_id]['location']['longitude']=konten[1]
        listData[chat_id]['location']['distance']=str(round(calc_distance(konten[0],konten[1]))) + " m"
        send_message("Location berhasil terinput",chat_id)
    except KeyError:
        send_message("Anda belum menginputkan nomor tiket",chat_id)
    return(listData)   
def cek_progres(listData, chat_id):
    print(listData)
    try:
        if((listData[chat_id]['location']['status']==True)and(listData[chat_id]['keterangan']['status']==True)and(listData[chat_id]['gambar']['sebelum']['status']==True)and(listData[chat_id]['gambar']['sesudah']['status']==True)):
            send_message("Input tiket dengan nomor ",listData[chat_id]["no_tiket"]," Selesai",chat_id)
            listData.pop(chat_id)
    except KeyError:
        True
    return(listData)
def main():
    tipekonten, konten, chat ,message = get_last_chat_id_and_text(get_updates())
    last_textchat = (konten, chat)
    data = defaultdict(dict)
    while True:
        tipekonten, konten, chat, message = get_last_chat_id_and_text(get_updates())
        if (konten, chat) != last_textchat:
            print((konten, chat), last_textchat)
            
            if(tipekonten=="text"):
                if(konten.find('/input_tiket')==0):
                    print(data)
                    data=input_tiket(data, konten, chat)
                    
                elif(konten.find('/input_keterangan')==0):
                    print(data)
                    data=input_keterangan(data, konten, chat)
            elif(tipekonten=="gambar"):
                print(data)
                if(konten[1].find('/input_gambar')==0):
                    data=input_gambar(data,konten,chat)
            elif(tipekonten=="location"):
                print(data)
                data=input_location(data,konten,chat)   
                
            last_textchat = (konten, chat)
            cek_progres(data,chat)
        time.sleep(0.5)


if __name__ == '__main__':
    main()


('/input_tiket inv232', 779884734) (['AgADBQADbqgxG10YCVRus_fsOEopLOxr3jIABJEFPkYg1FO1pgUGAAEC', None], 779884734)
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {'779884734': {'no_tiket': 'inv232', 'gambar': defaultdict(<class 'dict'>, {'sebelum': {'status': False, 'file_id': ''}, 'sesudah': {'status': False, 'file_id': ''}}), 'keterangan': defaultdict(<class 'dict'>, {'status': False, 'isi': ''}), 'location': defaultdict(<class 'dict'>, {'status': False, 'latitude': 0, 'longitude': 0, 'distance': 0}), 'statusTiket': False}})
(['AgADBQADbKkxGwUeCVSw1XvwoHEivf_w3zIABFApvNNf1ZonFYIEAAEC', '/input_gambar sebelum'], 779884734) ('/input_tiket inv232', 779884734)
defaultdict(<class 'dict'>, {'779884734': {'no_tiket': 'inv232', 'gambar': defaultdict(<class 'dict'>, {'sebelum': {'status': False, 'file_id': ''}, 'sesudah': {'status': False, 'file_id': ''}}), 'keterangan': defaultdict(<class 'dict'>, {'status': False, 'isi': ''}), 'location': defaultdict(<class 'dict'>, {'status': 

KeyboardInterrupt: 